In [1]:
import pandas as pd

In [2]:
file = pd.read_csv('genhousedata.csv'
                  )
df = file
df.head(15)

,brokered_by,status,price,bed,bath,acre_lot,street,city,state,zip_code,house_size,prev_sold_date
0,103378.0,for_sale,105000.0,3.0,2.0,0.12,1962661.0,Adjuntas,Puerto Rico,601.0,920.0,NaN
1,52707.0,for_sale,80000.0,4.0,2.0,0.08,1902874.0,Adjuntas,Puerto Rico,601.0,1527.0,NaN
2,103379.0,for_sale,67000.0,2.0,1.0,0.15,1404990.0,Juana Diaz,Puerto Rico,795.0,748.0,NaN
3,31239.0,for_sale,145000.0,4.0,2.0,0.10,1947675.0,Ponce,Puerto Rico,731.0,1800.0,NaN
4,34632.0,for_sale,65000.0,6.0,2.0,0.05,331151.0,Mayaguez,Puerto Rico,680.0,NaN,NaN
5,103378.0,for_sale,179000.0,4.0,3.0,0.46,1850806.0,San Sebastian,Puerto Rico,612.0,2520.0,NaN
6,1205.0,for_sale,50000.0,3.0,1.0,0.20,1298094.0,Ciales,Puerto Rico,639.0,2040.0,NaN
7,50739.0,for_sale,71600.0,3.0,2.0,0.08,1048466.0,Ponce,Puerto Rico,731.0,1050.0,NaN
8,81909.0,for_sale,100000.0,2.0,1.0,0.09,734904.0,Ponce,Puerto Rico,730.0,1092.0,NaN
9,65672.0,for_sale,300000.0,5.0,3.0,7.46,1946226.0,Las Marias,Puerto Rico,670.0,5403.0,NaN


# This is the original CSV. As you can see the fifth house among 568483 others are missing it's house size. 

In [4]:
df.isnull().sum()

brokered_by         4533
status                 0
price               1541
bed               481317
bath              511771
acre_lot          325589
street             10866
city                1407
state                  8
zip_code             299
house_size        568484
prev_sold_date    734297
dtype: int64

# Above are the missing values for all columns in this data set. Below the model begins. In the next cell the MAE, RMSE, and R2 scores are shown for model accuracy. 

In [6]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder
from lightgbm import LGBMRegressor

# Load data
df = pd.read_csv("genhousedata.csv")

features = ['price', 'bed', 'bath', 'acre_lot', 'city', 'state', 'zip_code', 'status']
target = 'house_size'

# Split data into with and without house_size
df_with_size = df[df[target].notnull()]
df_missing_size = df[df[target].isnull()]

# Drop rows with missing features in training data
train_data = df_with_size[features + [target]].dropna(subset=features)

# Calculate IQR to define outliers
Q1 = train_data[target].quantile(0.25)
Q3 = train_data[target].quantile(0.75)
IQR = Q3 - Q1
lower = Q1 - 1.5 * IQR
upper = Q3 + 1.5 * IQR

# Add outlier/normal flag column to the entire dataset (NaN for missing house_size)
def label_outlier(row):
    val = row[target]
    if pd.isna(val):
        return np.nan
    return 'outlier' if (val < lower or val > upper) else 'normal'

df['size_label'] = df.apply(label_outlier, axis=1)

# Preprocessing setup
numeric_features = ['price', 'bed', 'bath', 'acre_lot']
categorical_features = ['city', 'state', 'zip_code', 'status']

numeric_transformer = SimpleImputer(strategy='median')
categorical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='constant', fill_value='missing')),
    ('onehot', OneHotEncoder(handle_unknown='ignore', sparse_output=True))
])

preprocessor = ColumnTransformer([
    ('num', numeric_transformer, numeric_features),
    ('cat', categorical_transformer, categorical_features)
])

# Fit preprocessor on training data features
X_train = train_data[features]
y_train = train_data[target]
preprocessor.fit(X_train)

# Train only the normal model on the whole training set (including outliers if you want, but recommended just normal)
normal_train_data = train_data[(train_data[target] >= lower) & (train_data[target] <= upper)]
Xn_train = normal_train_data[features]
yn_train = normal_train_data[target]

normal_model = Pipeline([
    ('preprocessor', preprocessor),
    ('regressor', LGBMRegressor(n_estimators=100, random_state=42))
])

normal_model.fit(Xn_train, yn_train)

# Predict missing house_size values using normal_model only
X_missing = df_missing_size[features].copy()
preds = normal_model.predict(X_missing)

# Fill in missing house_size with predictions
df.loc[X_missing.index, target] = preds

# Optionally, update size_label for these newly predicted rows
def label_predicted_size(size):
    return 'normal' if (lower <= size <= upper) else 'outlier'

df.loc[X_missing.index, 'size_label'] = [label_predicted_size(s) for s in preds]

# Now df has missing house_size filled and a new column 'size_label' indicating normal/outlier

# You can save the updated dataframe if you want
#df.to_csv("genhousedata_filled.csv", index=False)

[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.025541 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 40618
[LightGBM] [Info] Number of data points in the train set: 1294270, number of used features: 20038
[LightGBM] [Info] Start training from score 1905.541743


In [7]:
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score

# === Add after fitting the model ===

# Create validation split from normal data
Xn_train_split, Xn_valid_split, yn_train_split, yn_valid_split = train_test_split(
    Xn_train, yn_train, test_size=0.2, random_state=42
)

# Train model on split training set
normal_model_split = Pipeline([
    ('preprocessor', preprocessor),
    ('regressor', LGBMRegressor(n_estimators=100, random_state=42))
])

normal_model_split.fit(Xn_train_split, yn_train_split)

# Predict on validation set
preds_valid = normal_model_split.predict(Xn_valid_split)

# Evaluate
mae = mean_absolute_error(yn_valid_split, preds_valid)
rmse = np.sqrt(mean_squared_error(yn_valid_split, preds_valid))
r2 = r2_score(yn_valid_split, preds_valid)

print("📊 Normal Model Evaluation on Validation Set:")
print(f"   MAE:  {mae:.2f} sq ft")
print(f"   RMSE: {rmse:.2f} sq ft")
print(f"   R²:   {r2:.4f}")

[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.070567 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 40618
[LightGBM] [Info] Number of data points in the train set: 1035416, number of used features: 20041
[LightGBM] [Info] Start training from score 1905.852493
📊 Normal Model Evaluation on Validation Set:
   MAE:  291.81 sq ft
   RMSE: 391.84 sq ft
   R²:   0.7293


# These accuracy metrics indicate that the predictions may vary by approximately ±300 square feet, which is roughly the size of a one person small private office. This level of error is acceptable for our purposes.

In [9]:
df.isnull().sum()

brokered_by         4533
status                 0
price               1541
bed               481317
bath              511771
acre_lot          325589
street             10866
city                1407
state                  8
zip_code             299
house_size             0
prev_sold_date    734297
size_label             0
dtype: int64

# Missing house sizes here have gone from 568484 to 0. All missing values were filled by the predictor!!!

In [11]:
# Round predicted house_size values to, say, 2 decimal places (change as needed)
df.loc[X_missing.index, target] = preds.round(0)  # Keep as float, just rounded

# Update size_label for predicted rows based on the rounded predictions
df.loc[X_missing.index, 'size_label'] = [label_predicted_size(s) for s in df.loc[X_missing.index, target]]

# Add a new column to flag predicted vs original
df['house_size_predicted'] = False  # Default: False for all original values
df.loc[X_missing.index, 'house_size_predicted'] = True  # Mark predicted rows as True

# The code above rounds the predicted house size values to the nearest whole number and creates a new column indicating whether each house size was predicted or originally present.

In [13]:
# df.to_csv("genhousedata_filled.csv", index=False)

In [14]:
norm_rows = df[(df['size_label'] == 'normal') & (df['house_size_predicted'] == True)]
norm_rows1 = df[(df['size_label'] == 'normal') & (df['house_size_predicted'] == False)]
norm_rows2 = df[(df['size_label'] == 'outlier') & (df['house_size_predicted'] == True)]
norm_rows3 = df[(df['size_label'] == 'outlier') & (df['house_size_predicted'] == False)]

print (f"{len(norm_rows1)} original rows with normal house sizes.")
print (f"{len(norm_rows)} predicted rows with normal house sizes.")
print (f"{len(norm_rows3)} original rows with outlier house sizes.")
print (f"{len(norm_rows2)} predicted rows with an outlier house size.")
print (f"{len(df)} total rows.")

1584831 original rows with normal house sizes.
568484 predicted rows with normal house sizes.
73067 original rows with outlier house sizes.
0 predicted rows with an outlier house size.
2226382 total rows.
